## Integrate the particles where they feel all NBody forces

In [1]:
import h5py
import tstrippy
from gcs import path_handler as ph
import gcs
import os 
import numpy as np 
import astropy.units as u
import vanilla
import vanilla_loop_over_monte_carlo as vlm
import datetime 

In [2]:
author = "Salvatore Ferrone"
author_affiliation = "Sapienza University of Rome"
author_email = "salvatore.ferrone@uniroma1.it"
description = "Integrating star-particles with in a globular cluster"

In [3]:
GCname              =   "NGC104"
montecarlokey       =   "monte-carlo-000"
internal_dynamics   =   "isotropic-plummer"
GCorbits_potential  =   "pouliasis2017pii"
MWpotential         =   "pouliasis2017pii"
NP                  =   int(1e2)
T                   =   5e9*u.yr
dt                  =   1e5*u.yr
NSKIP               =   int(100)

In [4]:
attrs = {"GCname":GCname,"montecarlokey":montecarlokey,"internal_dynamics":internal_dynamics,"GCorbits_potential":GCorbits_potential,"MWpotential":MWpotential,"NP":NP,"T":str(T),"dt":str(dt),"NSKIP":NSKIP}

In [5]:
T,dt,NSTEP,t_sampling=gcs.misc.get_time_sampling_from_years_to_integration_units(T,dt)

In [6]:
snapshotTime=t_sampling[::NSKIP]

In [7]:
outfilename=ph.StreamSnapShots(GCname,NP,MWpotential,internal_dynamics,montecarlokey)
tempdir=ph._TemporaryStreamSnapShots(MWpotential,GCname,NP,internal_dynamics,montecarlokey)

In [8]:
gcs.writers.Stream.assemble_StreamSnapShots(outfilename,snapshotTime,attrs,tempdir)

In [20]:
myfile = h5py.File(outfilename, 'r')

In [21]:
for att in myfile.attrs:
    print(att,myfile.attrs[att])

GCname NGC104
GCorbits_potential pouliasis2017pii
MWpotential pouliasis2017pii
NP 100
NSKIP 100
T 5000000000.0 yr
dt 100000.0 yr
internal_dynamics isotropic-plummer
montecarlokey monte-carlo-000


In [19]:
myfile.close()

# GCNBody stuff

load the host's Mass and size

In [6]:
Mass,rh_m,_a,_,_,_,_,_=gcs.extractors.MonteCarloObservables.extract_all_GC_observables([GCname],montecarlokey)
rplummer=gcs.misc.half_mass_to_plummer(rh_m[0]).value
mass_host = Mass[0].value

load all the perturbers 

In [7]:
GCnames=list(tstrippy.Parsers.baumgardtMWGCs().data['Cluster'][:])
GCnames.remove(GCname)

In [8]:
ts,xs,ys,zs,vxs,vys,vzs=gcs.extractors.GCOrbits.extract_orbits_from_all_GCS(GCnames,GCorbits_potential,montecarlokey)

In [9]:
Masses,rh_mes,RAes,DECes,Rsunes,RVes,mualphaes,mu_deltaes=gcs.extractors.MonteCarloObservables.extract_all_GC_observables(GCnames,montecarlokey)
r_plums = [gcs.misc.half_mass_to_plummer(rh_m).value for rh_m in rh_mes]
Masses = [Mass.value for Mass in Masses]

In [10]:
# load milky way parameters
MWparams = tstrippy.Parsers.potential_parameters.pouliasis2017pii()

In [11]:
integrator = tstrippy.integrator

In [12]:
integrator.setstaticgalaxy(MWpotential,MWparams)
integrator.setintegrationparameters(T.value,dt.value,Nstep)
integrator.setinitialkinematics(x+xH[0],y+yH[0],z+zH[0],vx+vxH[0],vy+vyH[0],vz+vzH[0])

In [13]:
integrator.inithostperturber(tnew,xH,yH,zH,vxH,vyH,vzH,mass_host,rplummer,)
integrator.initperturbers(ts,xs,ys,zs,Masses,r_plums)

In [15]:
integrator.leapfrogtofinalpositions()

In [17]:
xf=integrator.xf
yf=integrator.yf
zf=integrator.zf
vxf=integrator.vxf
vyf=integrator.vyf
vzf=integrator.vzf
tesc=integrator.tesc

In [18]:
import matplotlib.pyplot as plt